In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='ucsb-pt')

In [4]:
from tqdm import tqdm

# pull india boundary for filtering embeddings
india = ee.FeatureCollection("users/ptripathy/IND_Boundary")

# Filter and sort embeddings collection by system:index
embeddings = (ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL')
              .filterBounds(india)
              .filterDate('2024-01-01', '2025-01-01')
              .sort('system:index'))

total_images = embeddings.size().getInfo()
print('Total images for India 2024:', total_images)

# Export function
def export_image(img):
    name = ee.String(img.get('UTM_ZONE')).cat('_').cat(ee.String(img.get('system:index'))).getInfo()
    """
    task = ee.batch.Export.image.toDrive(
        image=img,
        description=name,
        folder='2025_IGUIDE_GoogleEmbeddings',
        region=img.geometry(),
        scale=30,
        crs=img.projection().crs(),
        fileDimensions=5120,
        maxPixels=1e13
    )
    """
    task = ee.batch.Export.image.toCloudStorage(
        image=img,
        description=name,
        bucket='iguide-onetb',  # Replace with your GCS bucket name
        fileNamePrefix='GoogleEmbeddings2024/' + name,
        region=img.geometry(),
        scale=30,
        crs=img.projection().crs(),
        fileDimensions=5120,
        maxPixels=1e13
    )

    task.start()

# Export images in batches by index range
start_idx = 111
end_idx = 115

image_list = embeddings.toList(embeddings.size())
for i in tqdm(range(start_idx, min(end_idx + 1, total_images))):
    export_image(ee.Image(image_list.get(i)))
print(f'Exported batch: images {start_idx} to {min(end_idx, total_images-1)}')

Total images for India 2024: 211


100%|██████████| 5/5 [00:46<00:00,  9.23s/it]

Exported batch: images 111 to 115


In [ ]:
# run this if you want to minotor the progress
import time

while True:
    tasks = ee.batch.Task.list()  # Refresh task list each loop

    pending = sum(1 for task in tasks if task.status()['state'] == 'READY')
    running = sum(1 for task in tasks if task.status()['state'] == 'RUNNING')
    completed = sum(1 for task in tasks if task.status()['state'] == 'COMPLETED')
    failed = sum(1 for task in tasks if task.status()['state'] == 'FAILED')

    print(f'Tasks - Pending: {pending}, Running: {running}, Completed: {completed}, Failed: {failed}')

    if pending == 0 and running == 0:
        print('All tasks finished!')
        break

    time.sleep(5)